## Attempts to selecting patents

### SQL Query for First Appearance of Each IPC/CPC Classification with Filing ID

In [2]:
import awswrangler as wr
!aws --version
!aws sso login

aws-cli/2.15.58 Python/3.11.8 Linux/5.11.0-1022-aws exe/x86_64.ubuntu.20
Attempting to automatically open the SSO authorization page in your default browser.
If the browser does not open or you wish to use a different device to authorize this request, open the following URL:

https://wipo.awsapps.com/start/#/device

Then enter the code:

DXJQ-FDPW
Successfully logged into Start URL: https://wipo.awsapps.com/start/


In [3]:
db='tmpdatabase' # when querying deda_ingestion_prd_link remember to set tmpdatabase for cache issues
sql='SELECT * FROM "landscaperdb"."landscaper_patstat_ext" limit 10;'
df = wr.athena.read_sql_query(sql=sql,database=db) 
df.head() 

,appln_id,appln_auth,appln_nr,appln_kind,appln_filing_date,appln_filing_year,appln_nr_original,ipr_type,receiving_office,internat_appln_id,...,load_date,appt_ctry_code,invt_ctry_code,appln_title_lg,appln_title,appln_abstract_lg,appln_abstract,ipc_class_symbol,ipc_position,cpc_class_symbol
0,21481872,GB,0518114,A,2005-09-06,2005,GB0518114,PI,,0,...,2025-05-15,[GB],"[GB, GB, GB, GB]",[en],[Ester slip agents],[en],[A composition comprises at least two esters h...,"[C08K 5/101, C08L 67/00]","[F, L]","[C08K 5/101, C08K2201/014, C08L 67/00]"
1,21481997,GB,0518240,A,2004-02-09,2004,GB0518240,PI,,54493580,...,2025-05-15,[US],"[US, US, US, US]",[en],[Downhole telemetry system using discrete mult...,[en],[A communication system usable in a borehole i...,"[E21B 47/12, G01V 3/00, G01V 3/18, H04H ...","[ , L, F, L, L]","[E21B 47/13, G01V 3/00, G01V 3/18, H04B ..."
2,21482023,GB,0518266,A,2005-09-08,2005,GB0518266,PI,,0,...,2025-05-15,[ ],None,[en],[Electrotherapy apparatus & method],None,None,None,None,None
3,21481404,GB,0517639,A,2005-08-30,2005,GB0517639,PI,,0,...,2025-05-15,[ ],None,[en],[Lined waste receptacles],None,None,None,None,None
4,21481680,GB,0517917,A,2005-09-03,2005,GB0517917,PI,,0,...,2025-05-15,[ ],None,[en],[A bait box],None,None,"[A01M 23/24, A01M 23/30, A01M 31/00]","[ , , ]","[A01M 23/245, A01M 23/30, A01M 31/002]"


## SQL Query Explanation: First Appearance of Each IPC/CPC Classification

In [4]:
sql="""

 WITH RankedFilings AS (
    SELECT 
        SUBSTR(earliest_filing_date, 1, 4) AS filing_year,
        fam_ipc_cpc,
        ROW_NUMBER() OVER (PARTITION BY fam_ipc_cpc ORDER BY earliest_filing_date) AS rn
    FROM 
        "landscaperdb"."landscaper_family_ext"
)
SELECT 
    filing_year AS first_appearance_year,
    fam_ipc_cpc
FROM 
    RankedFilings
WHERE 
    rn = 1
ORDER BY 
    first_appearance_year,
    fam_ipc_cpc"""

db='tmpdatabase' # when querying deda_ingestion_prd_link remember to set tmpdatabase for cache issues
df = wr.athena.read_sql_query(sql=sql,database=db) 
df.head()

,first_appearance_year,fam_ipc_cpc
0,1933,"[B65F 1/02, B65F2240/14, Y10S 242/905]"
1,1933,"[B65F 1/06, B65F 1/10, B65F 7/00, Y10S 2..."
2,1933,"[B65F 1/08, B65F 1/16, G09F 23/00, B65F ..."
3,1933,"[B65F 1/14, B65F 1/1436, B65F2001/1669, B6..."
4,1933,"[B65F 1/16, H01R 4/50, H01R 4/5033]"


In [ ]:
sql="""

WITH RankedFilings AS (
    SELECT 
        SUBSTR(earliest_filing_date, 1, 4) AS filing_year,
        fam_ipc_cpc,
        ROW_NUMBER() OVER (PARTITION BY fam_ipc_cpc ORDER BY earliest_filing_date) AS rn
    FROM 
        "landscaperdb"."landscaper_family_ext"
)
SELECT 
    filing_year AS first_appearance_year,
    fam_ipc_cpc
FROM 
    RankedFilings
WHERE 
    rn = 1
ORDER BY
    filing_year,
    fam_ipc_cpc"""

db='tmpdatabase' # when querying deda_ingestion_prd_link remember to set tmpdatabase for cache issues
df = wr.athena.read_sql_query(sql=sql,database=db) 
df.head()

### example Wind 

In [5]:
sql="""

WITH RankedFilings AS (
    SELECT 
        SUBSTR(earliest_filing_date, 1, 4) AS filing_year,
        earliest_filing_id,
        fam_ipc_cpc,
        ROW_NUMBER() OVER (PARTITION BY fam_ipc_cpc ORDER BY earliest_filing_date) AS rn
    FROM 
         "landscaperdb"."landscaper_family_ext"
    WHERE
        any_match (fam_ipc_cpc, x -> x LIKE '%F03D%')
)
SELECT 
    filing_year AS first_appearance_year,
    earliest_filing_id,
    fam_ipc_cpc
FROM 
    RankedFilings
WHERE 
    rn = 1
ORDER BY 
    first_appearance_year,
    fam_ipc_cpc

 """

db='tmpdatabase' # when querying deda_ingestion_prd_link remember to set tmpdatabase for cache issues
df = wr.athena.read_sql_query(sql=sql,database=db) 
df

,first_appearance_year,earliest_filing_id,fam_ipc_cpc
0,2010,363744069,"[F03D 9/00, F24J 2/00, Y02E 10/40, Y02E ..."
1,2010,352141189,"[F03D 9/00, F24J 2/40, F03D 1/02, F03D ..."
2,2010,375119417,"[F03D 9/00, F24J 2/46, Y02E 10/40, Y02E ..."
3,2010,337004832,"[F03D 9/00, F24J 2/54, F03D 9/007, F05B2..."
4,2010,334071744,"[F03D 9/00, F24S 20/30, H01L 31/058, F03D ..."
...,...,...,...
84998,2021,555331727,"[F03D 7/04, F03D 9/20, F03D 15/00, F03D ..."
84999,2021,555335611,"[F03D 7/04, F03D 9/25, F03D 7/0204, F03D..."
85000,2021,559654746,"[F03D 7/04, F03D 9/25, F03D 7/0284, F03D..."
85001,2021,553227891,"[F03D 7/04, F03D 9/25, F03D 9/32, F03D ..."


In [11]:
sql="""

WITH RankedFilings AS (
    SELECT 
        SUBSTR(earliest_filing_date, 1, 4) AS filing_year,
        earliest_filing_id,
        fam_ipc_cpc,
        ROW_NUMBER() OVER (ORDER BY earliest_filing_date) AS rn
    FROM 
        "landscaperdb"."landscaper_family_ext"
    WHERE
        any_match (fam_ipc_cpc, x -> x LIKE '%F03D%') 
)
SELECT 
    filing_year AS first_appearance_year,
    earliest_filing_id,
    fam_ipc_cpc
FROM 
    RankedFilings
WHERE 
    rn = 1
 """

db='tmpdatabase' # when querying deda_ingestion_prd_link remember to set tmpdatabase for cache issues
df = wr.athena.read_sql_query(sql=sql,database=db) 
df

,first_appearance_year,earliest_filing_id,fam_ipc_cpc
0,1882,46015236,"[F03D 15/00, F03D 15/10, F03D 80/70, F05B22..."


### Example Oncomouse

In [6]:
sql="""
WITH RankedFilings AS (
    SELECT 
        SUBSTR(earliest_filing_date, 1, 4) AS filing_year,
        earliest_filing_id,
        fam_ipc_cpc,
        ROW_NUMBER() OVER (ORDER BY earliest_filing_date) AS rn
    FROM 
        "landscaperdb"."landscaper_family_ext"
    WHERE
        any_match (fam_ipc_cpc, x -> x LIKE '%A01K  67%') AND 
        any_match (fam_ipc_cpc, x -> x LIKE '%C07H  21%')
)
SELECT 
    filing_year AS first_appearance_year,
    earliest_filing_id,
    fam_ipc_cpc
FROM 
    RankedFilings
WHERE 
    rn = 1
 """

db='tmpdatabase' # when querying deda_ingestion_prd_link remember to set tmpdatabase for cache issues
df = wr.athena.read_sql_query(sql=sql,database=db) 
df

,first_appearance_year,earliest_filing_id,fam_ipc_cpc
0,1984,52301033,"[A01K 67/00, A01K 67/02, A01K 67/027, A01K ..."
